In [3]:
%matplotlib inline

import os
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.graph_objects as go


In [ ]:
import os
import pandas as pd
import plotly.graph_objects as go


def plot_aggregated_actual_vs_predicted(
    actual, predicted, lookback, forecast_horizon, percentage=100, save_path=None
):
    """
    Plots actual vs. predicted values following the correct aggregation:
    - First `lookback + forecast_horizon` values are kept.
    - The last predicted value from each step-ahead sequence is progressively added.
    - Allows plotting only a percentage of the last rows.

    :param actual: List or Pandas Series of actual values
    :param predicted: List or Pandas Series of predicted values
    :param lookback: Lookback window size
    :param forecast_horizon: Forecast horizon size
    :param percentage: Percentage of last samples to show (default 100%)
    :param save_path: If provided, saves the plot as a high-resolution PNG
    """

    num_samples = len(actual)
    plot_samples = (
        41  # int(num_samples * (percentage / 100)) if num_samples > 30 else num_samples
    )

    # Ensure actual and predicted are lists
    actual = actual.tolist() if isinstance(actual, pd.Series) else list(actual)
    predicted = (
        predicted.tolist() if isinstance(predicted, pd.Series) else list(predicted)
    )

    # Initialize aggregated lists
    aggregated_actual = actual[: lookback + forecast_horizon]
    aggregated_predicted = predicted[: lookback + forecast_horizon]

    # Add last step of each forecast progressively
    for i in range(lookback + forecast_horizon, num_samples, forecast_horizon):
        aggregated_actual.append(actual[i])
        aggregated_predicted.append(predicted[i])

    # Select only the last `plot_samples` samples for visualization
    aggregated_actual = aggregated_actual[-plot_samples:]
    aggregated_predicted = aggregated_predicted[-plot_samples:]

    # Create Plotly figure
    fig = go.Figure()

    # Add actual values
    fig.add_trace(
        go.Scatter(
            y=aggregated_actual,
            mode="lines",
            name="Actual",
            line=dict(color="blue", width=8),  # Thicker lines
        )
    )

    # Add predicted values
    fig.add_trace(
        go.Scatter(
            y=aggregated_predicted,
            mode="lines",
            name="Predicted",
            line=dict(color="orange", dash="dash", width=8),  # Thicker dashed line
        )
    )

    # Customize layout with bigger font sizes and adjusted resolution
    fig.update_layout(
        title=dict(
            text=f"{lookback} lookback days / {forecast_horizon} days ahead - Actual vs Predicted ({plot_samples-1} last Samples)",
            font=dict(size=26, family="Arial", weight="bold"),  # Bigger title
        ),
        xaxis=dict(
            title="Samples",
            titlefont=dict(size=26, family="Arial", weight="bold"),
            tickfont=dict(size=26, family="Arial", weight="bold"),
        ),
        yaxis=dict(
            title="Power Consumption (KWh)",
            titlefont=dict(size=26, family="Arial", weight="bold"),
            tickfont=dict(size=26, family="Arial", weight="bold"),
        ),
        legend=dict(
            font=dict(size=26, family="Arial", weight="bold"),
        ),
        template="plotly_white",
        autosize=False,
        width=1100,  # Increased width for better readability
        height=750,  # Increased height for better readability
    )

    # Show the figure inline (useful for Jupyter notebooks)
    fig.show()

    # Save high-resolution image if save_path is provided
    if save_path:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        fig.write_image(
            f"{save_path}.png", scale=10
        )  # Scale increases resolution & DPI
        fig.write_image(f"{save_path}.pdf", scale=10)  # Save as vector format
        print(f"Aggregated plot saved to {save_path}")


## Lookback 30 --> forecast 7


In [37]:
import os
import pandas as pd

# Define file paths
file_paths = [
    "./30_xx/ihpc/30_7.csv",
    "./30_xx/ihpc/30_14.csv",
    "./30_xx/ihpc/30_21.csv",
    "./30_xx/ihpc/30_30.csv",
    "./30_xx/ihpc/30_60.csv",
    "./30_xx/ihpc/30_90.csv",
    #--------------------
    "./30_xx/eld/30_7.csv",
    "./30_xx/eld/30_14.csv",
    "./30_xx/eld/30_21.csv",
    "./30_xx/eld/30_30.csv",
    "./30_xx/eld/30_60.csv",
    "./30_xx/eld/30_90.csv",
]

# Loop through each file and process
for file_path in file_paths:
    try:
        # Extract subdirectory (ihpc or eld) and forecast horizon from filename
        parts = file_path.split("/")
        subdir = parts[-2]  # Extract ihpc or eld
        forecast_horizon = int(parts[-1].split("_")[1].split(".")[0])  # Extract 7, 14, etc.

        print(f"Processing file: {file_path} | Subdir: {subdir} | Forecast Horizon: {forecast_horizon}")

        # Load data
        df = pd.read_csv(file_path)

        # Ensure correct column names
        df.columns = ["Actual", "Predicted"]

        # Define save path inside correct subdirectory
        save_path = f"./30_xx/images/{subdir}/30_{forecast_horizon}"
        os.makedirs(os.path.dirname(save_path), exist_ok=True)  # Ensure folder exists

        # Call the plotting function
        plot_aggregated_actual_vs_predicted(
            actual=df["Actual"],
            predicted=df["Predicted"],
            lookback=30,
            forecast_horizon=forecast_horizon,
            percentage=25,
            save_path=save_path
        )

        print(f"✅ Saved plot to: {save_path}.png\n")

    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")


Processing file: ./30_xx/ihpc/30_7.csv | Subdir: ihpc | Forecast Horizon: 7


Aggregated plot saved to ./30_xx/images/ihpc/30_7
✅ Saved plot to: ./30_xx/images/ihpc/30_7.png

Processing file: ./30_xx/ihpc/30_14.csv | Subdir: ihpc | Forecast Horizon: 14


Aggregated plot saved to ./30_xx/images/ihpc/30_14
✅ Saved plot to: ./30_xx/images/ihpc/30_14.png

Processing file: ./30_xx/ihpc/30_21.csv | Subdir: ihpc | Forecast Horizon: 21


Aggregated plot saved to ./30_xx/images/ihpc/30_21
✅ Saved plot to: ./30_xx/images/ihpc/30_21.png

Processing file: ./30_xx/ihpc/30_30.csv | Subdir: ihpc | Forecast Horizon: 30


Aggregated plot saved to ./30_xx/images/ihpc/30_30
✅ Saved plot to: ./30_xx/images/ihpc/30_30.png

Processing file: ./30_xx/ihpc/30_60.csv | Subdir: ihpc | Forecast Horizon: 60


...

Aggregated plot saved to ./30_xx/images/eld/30_14
✅ Saved plot to: ./30_xx/images/eld/30_14.png

Processing file: ./30_xx/eld/30_21.csv | Subdir: eld | Forecast Horizon: 21


Aggregated plot saved to ./30_xx/images/eld/30_21
✅ Saved plot to: ./30_xx/images/eld/30_21.png

Processing file: ./30_xx/eld/30_30.csv | Subdir: eld | Forecast Horizon: 30


Aggregated plot saved to ./30_xx/images/eld/30_30
✅ Saved plot to: ./30_xx/images/eld/30_30.png

Processing file: ./30_xx/eld/30_60.csv | Subdir: eld | Forecast Horizon: 60


Aggregated plot saved to ./30_xx/images/eld/30_60
✅ Saved plot to: ./30_xx/images/eld/30_60.png

Processing file: ./30_xx/eld/30_90.csv | Subdir: eld | Forecast Horizon: 90


Aggregated plot saved to ./30_xx/images/eld/30_90
✅ Saved plot to: ./30_xx/images/eld/30_90.png

